<a href="https://colab.research.google.com/github/leadeeerq/data-science/blob/main/230227_DataCleaningIdentifyingHiddenMissingValues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

##Loading the data

In [45]:
df = pd.read_csv('/content/drive/MyDrive/230222_Master 12 Advanced Projects/Project_03_Materials/movies_metadata.csv', low_memory = False)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

###Stringified data

In [46]:
df.genres[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [47]:
df.belongs_to_collection[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

###Dropping irrelevant columns

In [9]:
df.adult.value_counts()

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: adult, dtype: int64

In [48]:
df.drop(columns = ['adult'], inplace = True)

In [49]:
df.drop(columns = ['imdb_id', 'original_title', 'video', 'homepage'], inplace = True)

##Handling stringified JSON columns, ast library

In [50]:
import json
import ast

In [51]:
json_cols = ['belongs_to_collection', 'genres', 'production_countries', 'production_companies', 'spoken_languages']

In [14]:
df.belongs_to_collection[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

###Replacing single quotes with double ' -> "

In [15]:
example1 = "{'dog': 3, 'cat': 5}"

In [16]:
json.loads(example1)

JSONDecodeError: ignored

In [18]:
example2 = '{"dog": 3, "cat": 5}'
json.loads(example2)

{'dog': 3, 'cat': 5}

In [19]:
json.loads(example1.replace("'", '"'))

{'dog': 3, 'cat': 5}

###That leads to having a list

In [20]:
df.genres.apply(lambda x: json.loads(x.replace("'", '"')))[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

###Using ast.literal_eval - it evaluates a string containing a python expression

In [23]:
ast.literal_eval(example1)

{'dog': 3, 'cat': 5}

In [22]:
ast.literal_eval(example2)

{'dog': 3, 'cat': 5}

In [24]:
df.genres.apply(ast.literal_eval)[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [27]:
df.genres = df.genres.apply(ast.literal_eval)

###Applying the ast.literal_eval function to all columns

It doesn't work for values different than strings

In [28]:
df.loc[:, json_cols].apply(ast.literal_eval, axis = 0)

ValueError: ignored

In [32]:
df.belongs_to_collection.apply(lambda x: isinstance(x, str))

0         True
1        False
2         True
3        False
4         True
         ...  
45461    False
45462    False
45463    False
45464    False
45465    False
Name: belongs_to_collection, Length: 45466, dtype: bool

In [37]:
import numpy as np

In [38]:
df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [39]:
df.belongs_to_collection

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object

In [52]:
for col in json_cols:
  df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [53]:
df

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,fa,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,4.0,1.0
45462,NaN,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3.0
45463,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,3.8,6.0
45464,NaN,0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,0.0,0.0


##Flattening the nested columns

In [54]:
df.belongs_to_collection[0]

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

###Belongs_to_collection - take only the value of the key 'name'

In [55]:
df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

In [56]:
df.belongs_to_collection.value_counts(dropna = False).head(10)

NaN                                       40975
The Bowery Boys                              29
Totò Collection                              27
James Bond Collection                        26
Zatôichi: The Blind Swordsman                26
The Carry On Collection                      25
Pokémon Collection                           22
Charlie Chan (Sidney Toler) Collection       21
Godzilla (Showa) Collection                  16
Uuno Turhapuro                               15
Name: belongs_to_collection, dtype: int64

###Genres - join values of keys 'name' with '|'

In [57]:
df.genres[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [58]:
df.genres = df.genres.apply(lambda x: '|'.join(i['name'] for i in x))

In [59]:
df.genres[0]

'Animation|Comedy|Family'

In [60]:
df.genres.value_counts(dropna = False).head(10)

Drama                   5000
Comedy                  3621
Documentary             2723
                        2442
Drama|Romance           1301
Comedy|Drama            1135
Horror                   974
Comedy|Romance           930
Comedy|Drama|Romance     593
Drama|Comedy             532
Name: genres, dtype: int64

###Replacing empty strings with np.nan

In [ ]:
df.genres.replace('', np.nan, inplace = True)

###Spoken_languages, production_countries, production_companies same as genres

In [61]:
df.spoken_languages

0                 [{'iso_639_1': 'en', 'name': 'English'}]
1        [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
2                 [{'iso_639_1': 'en', 'name': 'English'}]
3                 [{'iso_639_1': 'en', 'name': 'English'}]
4                 [{'iso_639_1': 'en', 'name': 'English'}]
                               ...                        
45461               [{'iso_639_1': 'fa', 'name': 'فارسی'}]
45462                    [{'iso_639_1': 'tl', 'name': ''}]
45463             [{'iso_639_1': 'en', 'name': 'English'}]
45464                                                   []
45465             [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, Length: 45466, dtype: object

In [62]:
df.spoken_languages = df.spoken_languages.apply(lambda x: '|'.join(i['name'] for i in x) if isinstance(x, list) else np.nan)

In [63]:
df.spoken_languages

0                 English
1        English|Français
2                 English
3                 English
4                 English
               ...       
45461               فارسی
45462                    
45463             English
45464                    
45465             English
Name: spoken_languages, Length: 45466, dtype: object

In [64]:
df.spoken_languages.value_counts(dropna = False).head(10)

English             22395
                     3952
Français             1853
日本語                  1289
Italiano             1218
Español               902
Pусский               807
Deutsch               762
English|Français      681
English|Español       572
Name: spoken_languages, dtype: int64

In [65]:
df.spoken_languages.replace('', np.nan, inplace = True)

In [66]:
df.production_companies = df.production_companies.apply(lambda x: '|'.join(i['name'] for i in x) if isinstance(x, list) else np.nan)
df.production_countries = df.production_countries.apply(lambda x: '|'.join(i['name'] for i in x) if isinstance(x, list) else np.nan)

In [67]:
df.production_companies.replace('', np.nan, inplace = True)
df.production_countries.replace('', np.nan, inplace = True)

###Number of missing values - isna().sum()

In [68]:
df.isna().sum()

belongs_to_collection    40975
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
poster_path                386
production_companies     11881
production_countries      6288
release_date                87
revenue                      6
runtime                    263
spoken_languages          3958
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
dtype: int64

In [69]:
pd.read_csv('/content/drive/MyDrive/230222_Master 12 Advanced Projects/Project_03_Materials/movies_metadata.csv', low_memory = False).isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64